In [1]:
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import glob
import pandas as pd

NameCode_DF = pd.read_csv("requiredTeacherCodes.csv")##file where all teacher codes are stored to build the url
nameList = NameCode_DF["Name"].tolist()
codeNumber = NameCode_DF["Code"].tolist()

numCodes = len(codeNumber)

for i in range(0,numCodes):
    is_LoadMore = True
    time.sleep(5)
    url_builder = "https://www.ratemyprofessors.com/ShowRatings.jsp?tid=" + str(codeNumber[i]) ##url to search to


    driver = webdriver.Safari()

    driver.get(url=url_builder)
    ##gives time to click out of the cookies button 
    time.sleep(5)
    try:
        loadMore = driver.find_element_by_css_selector("#loadMore")
        
    except NoSuchElementException: ##in case that page has no load button
        is_LoadMore = False
        
    if is_LoadMore:
        while(loadMore.is_displayed()):
            print("loadedMore")
            driver.execute_script("arguments[0].click()",loadMore) ##javascript necessary here to handle overlay that becomes present 
            time.sleep(5)
            loadMore = driver.find_element_by_css_selector("#loadMore")



    
    all_elem = driver.find_elements_by_class_name("descriptor-container") ##contains all rating information
    class_elem = driver.find_elements_by_class_name("response")##contains all class information
    
    classElem_list = []
    allElem_list = []

    for elements1 in all_elem:

        allElem_list.append(elements1.text)##contains rating information

    for elements in class_elem:

        classElem_list.append(elements.text)


    Overall_Quality = []
    Level_of_Difficulty = []

    for entireList in allElem_list:

        each_element = entireList.replace('\n','').strip().split(" ",maxsplit =1) ##formating to get relevant infromation
        rating_category = each_element[1].strip()
        overall_rating = each_element[0]

        if rating_category == "Level of Difficulty":
            Level_of_Difficulty.append(overall_rating)

        else:

            Overall_Quality.append(overall_rating)

    
    ##data formatting to get relevant data
    
    classes = classElem_list[::6]
    grades = classElem_list[5:]
    grades = grades[::6]

    DataFrame_toReturn = pd.DataFrame()
    DataFrame_toReturn['Level of Difficulty'] = Level_of_Difficulty
    DataFrame_toReturn['Overall Quality'] = Overall_Quality
    DataFrame_toReturn['Course'] = classes
    DataFrame_toReturn['Grade Received'] = grades
    
    ##DataFrame_toReturn.to_csv("teacherData/" + nameList[i] + ".csv",index=0)

    driver.quit()
    

In [2]:
import os

path, dirs, files = next(os.walk("teacherData"))


In [8]:

dataFrameVertical = pd.DataFrame()
# Print png images in folder C:\Users\admin\
for filepath in glob.iglob("teacherData/*.csv"):
    name = pd.read_csv(filepath)
    teacherName = os.path.split(filepath)
    distinctName = teacherName[1].split(".")[0]
    name = name.assign(Instructor = distinctName)
   
    dataFrameVertical = pd.concat([dataFrameVertical, name], axis=0)
    




In [10]:
dataFrameVertical.to_csv("CombinedRMP_List.csv")